In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
from multiversx_sdk import Address, Transaction, TransactionsFactoryConfig, ProxyNetworkProvider, ApiNetworkProvider
from tools.notebooks import env
import config

builder_config = TransactionsFactoryConfig(chain_id="1")
proxy_mainnet = ProxyNetworkProvider("https://gateway.multiversx.com")
api_mainnet = ApiNetworkProvider("https://api.multiversx.com")
proxy_shadowfork = ProxyNetworkProvider(config.DEFAULT_PROXY)

In [ ]:
from multiversx_sdk import UserSigner, Account
from pathlib import Path

user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))

In [ ]:
def prep_forward_tx(tx_hash: str) -> Transaction:
    original_tx_data = proxy_mainnet.get_transaction(tx_hash)
    print(original_tx_data.data)
    forwarded_tx  = Transaction(
        sender=original_tx_data.sender,
        receiver=original_tx_data.receiver,
        value=original_tx_data.value,
        data=original_tx_data.data,
        chain_id=builder_config.chain_id,
        gas_limit=original_tx_data.gas_limit,
        gas_price=original_tx_data.gas_price
    )

    print(forwarded_tx.to_dictionary())
    return forwarded_tx


def sign_and_send(tx: Transaction, account: UserSigner):
    tx.nonce = proxy_shadowfork.get_account(tx.sender).nonce
    tx.signature = account.sign(tx)
    proxy_shadowfork.send_transaction(tx)

clone & forward transaction

In [ ]:
hash = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

forwarded_tx = prep_forward_tx(hash)

In [ ]:
sign_and_send(forwarded_tx, user_signer)

clone & forward transactions on account

In [ ]:
from time import sleep
from multiversx_sdk.network_providers.http_resources import transaction_from_api_response, TransactionOnNetwork
contract_address = "erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg"

raw_transactions = api_mainnet.do_get_generic(f"accounts/{contract_address}/transactions")
transactions: list[TransactionOnNetwork] = []
for raw_tx in raw_transactions:
    transactions.append(transaction_from_api_response(raw_tx.get('txHash'), raw_tx))

for transaction in transactions:
    print(f"Forwarded tx hash: {transaction.hash}")
    forwarded_tx = prep_forward_tx(transaction.hash)
    sign_and_send(forwarded_tx, user_signer)
    sleep(1)